In [ ]:
import cv2
import dlib
import numpy as np

In [ ]:
PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(PREDICTOR_PATH)
detector = dlib.get_frontal_face_detector()

In [ ]:
# define a dictionary that maps the indexes of the facial
# landmarks to specific face regions
LM = dict({
	"mouth_outer": (48, 59),
	"mouth_inner": (60, 67),
	"mouth": (48, 68),
	"right_eyebrow": (17, 22),
	"left_eyebrow": (22, 27),
	"right_eye": (36, 42),
	"left_eye": (42, 48),
	"nose": (27, 35),
	"jaw": (0, 17)
})


In [ ]:
def get_landmarks(im):
    rects = detector(im, 1)
    '''
    if len(rects) > 1:
        raise "Error"
    if len(rects) == 0:
        raise "Error"
    '''    
    if len(rects) > 1:
        print("Error1")
    if len(rects) == 0:
        print("Error0")
    print(rects[0])
    return np.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])

def annotate_landmarks(im, landmarks):
    im = im.copy()
    
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(im, str(idx), pos, fontFace = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                    fontScale = 0.4, color = (0, 0, 255))
        cv2.circle(im, pos, 3, color = (0, 255, 255))
        
    return im

def top_lip(landmarks):
    top_lip_pts = []
    
    for i in range(50, 53):
        top_lip_pts.append(landmarks[i])
    
    for i in range(61, 64):
        top_lip_pts.append(landmarks[i])
    
    top_lip_all_pts = np.squeeze(np.asarray(top_lip_pts))
    #print(top_lip_all_pts[0][0])
    top_lip_mean = np.mean(top_lip_pts, axis = 0)
    return int(top_lip_mean[:,1])

def bottom_lip(landmarks):
    bottom_lip_pts = []
    
    for i in range(65, 68):
        bottom_lip_pts.append(landmarks[i])
    for i in range(56, 59):
        bottom_lip_pts.append(landmarks[i])
    
    bottom_lip_all_pts = np.squeeze(np.asarray(bottom_lip_pts))
    bottom_lip_mean = np.mean(bottom_lip_pts, axis = 0)
    return int(bottom_lip_mean[:,1])
 
def mouth_open(image):
    landmarks = get_landmarks(image)
    
    #if landmarks == "Error":
     #   return image, 0
    
    image_with_landmarks = annotate_landmarks(image, landmarks)
    top_lip_center = top_lip(landmarks)
    bottom_lip_center = bottom_lip(landmarks)
    lip_distance = abs(top_lip_center - bottom_lip_center)
    return image_with_landmarks, lip_distance

cap = cv2.VideoCapture(0)
yawns = 0
yawn_status = False

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, flipCode=1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    image_landmarks, lip_distance = mouth_open(gray)
    
    prev_yawn_status = yawn_status
    
    if lip_distance > 40:
        yawn_status = True
        
        cv2.putText(frame, "Subject is Yawning", (50, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0, 255), 2)
        
        output_text = "Yawn Count: " + str(yawns + 1)
        
        cv2.putText(frame, output_text, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 127), 2)
        
    else:
        yawn_status = False
        
    if prev_yawn_status == True and yawn_status == False:
        yawns += 1
        
    cv2.imshow('Live Landmarks', image_landmarks)
    cv2.imshow('Yawn Detection', frame)
    
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()